# Module 05 — Semiconductor Zoo
# Notebook 03: Reading an IC Datasheet
## DATASHEET DRILL — LM358 Op-Amp Walkthrough

---

Reading datasheets is the most important skill in electronics engineering. Every
component you use has a datasheet, and the ability to quickly extract the information
you need separates confident builders from confused ones.

In this notebook we walk through the **LM358 dual operational amplifier** datasheet
section by section, translating datasheet-speak into practical understanding.

---
## What is an Op-Amp?

An **operational amplifier** (op-amp) is a high-gain differential amplifier IC.
It has:

- **Two inputs**: non-inverting (+) and inverting (-)
- **One output**: $V_{out} = A_{OL} \times (V_+ - V_-)$
- **Very high open-loop gain**: $A_{OL}$ is typically 100,000 to 1,000,000

Because the gain is so high, op-amps are almost always used with **negative feedback**
to create stable, predictable circuits.

```
          +Vcc
           │
      ┌────┴────┐
  V+──┤+       │
      │  Op-Amp ├──── Vout
  V-──┤-       │
      └────┬────┘
           │
          GND (or -Vcc)
```

The LM358 is a **dual** op-amp (two op-amps in one DIP-8 package) that can operate
from a **single supply** (important — many op-amps need dual +/- supplies).

---
## Section 1: Pin Configuration

The LM358 comes in a DIP-8 package. Here is the pinout:

```
        ┌────U────┐
  1OUT ─┤1       8├─ Vcc
  1IN- ─┤2       7├─ 2OUT
  1IN+ ─┤3       6├─ 2IN-
   GND ─┤4       5├─ 2IN+
        └─────────┘
        
  Pin 1: Output A        Pin 8: V+ (supply)
  Pin 2: Inverting A     Pin 7: Output B
  Pin 3: Non-Inv A      Pin 6: Inverting B
  Pin 4: GND (V-)       Pin 5: Non-Inv B
```

**Reading tip:** The "U" at the top represents the notch on the physical IC.
Pin 1 is always to the left of the notch. Pin numbers go counter-clockwise.

---
## Section 2: Absolute Maximum Ratings

These are the **do not exceed** values. Exceeding any of these may permanently
damage the device.

| Parameter | LM358 Value | What it means |
|-----------|------------|---------------|
| Supply voltage (Vcc) | 32 V (single) | Max power supply voltage |
| Differential input voltage | +/- 32 V | Max voltage between + and - inputs |
| Input voltage range | -0.3 V to Vcc+0.3 V | Inputs must stay within this range |
| Output short-circuit duration | Continuous | Output can be shorted to GND (current limited) |
| Operating temperature | 0 to 70 C (commercial) | |
| Storage temperature | -65 to 150 C | |

**Key takeaway:** The LM358 is robust. 32V supply is generous for most hobbyist work.
The continuous short-circuit rating means you won't fry it if you accidentally
short the output — the internal current limiting protects it.

---
## Section 3: Electrical Characteristics

This is the heart of the datasheet. Let us go through each important parameter.

### Input Offset Voltage ($V_{OS}$)

**Typical: 2 mV, Max: 7 mV**

In a perfect op-amp, if both inputs are at the same voltage, the output would be
exactly mid-supply. In reality, internal transistor mismatches create a small
offset. This means $V_{out} = A_{OL} \times (V_+ - V_- + V_{OS})$.

For high-gain circuits, 7 mV of offset amplified by 100 = 0.7V of output error.
For precision work, use a "precision" op-amp with lower $V_{OS}$.

### Input Bias Current ($I_B$)

**Typical: 20 nA, Max: 150 nA**

The op-amp inputs draw a small current. The LM358 uses BJT input transistors,
so it needs nanoamps of bias current. (JFET or CMOS input op-amps draw picoamps.)

This matters when your source impedance is high: $V_{error} = I_B \times R_{source}$.
At 150 nA with a 1M source: $V_{error} = 150$ mV — significant!

### Open-Loop Gain ($A_{OL}$)

**Typical: 100 dB = 100,000 V/V**

This is the gain with no feedback. At DC, the LM358 amplifies the difference
between its inputs by 100,000. With negative feedback, the closed-loop gain
is set by external resistors and is much lower (and stable).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- LM358 Open-Loop Gain vs Frequency (Bode Plot) ---

# LM358 has a gain-bandwidth product of ~1 MHz and a dominant pole at ~10 Hz
GBW = 1e6            # 1 MHz gain-bandwidth product
A_OL_DC = 100_000    # 100 dB at DC
f_pole = GBW / A_OL_DC  # dominant pole frequency (~10 Hz)

f = np.logspace(-1, 7, 1000)  # 0.1 Hz to 10 MHz

# Single-pole model: A(f) = A_DC / sqrt(1 + (f/f_pole)^2)
A = A_OL_DC / np.sqrt(1 + (f / f_pole)**2)
A_dB = 20 * np.log10(A)

# Phase: -arctan(f/f_pole)
phase = -np.degrees(np.arctan(f / f_pole))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Magnitude plot
ax1.semilogx(f, A_dB, 'b-', linewidth=2, label='Open-loop gain')

# Mark key points
ax1.axhline(y=0, color='black', linewidth=0.5)
ax1.axhline(y=100, color='gray', linestyle='--', alpha=0.3)
ax1.axhline(y=40, color='gray', linestyle='--', alpha=0.3)
ax1.axhline(y=20, color='gray', linestyle='--', alpha=0.3)

# Show closed-loop gain lines
for gain_dB, label, color in [(40, 'Gain=100 (40dB)', '#FF6600'),
                                (20, 'Gain=10 (20dB)', '#009900'),
                                (0, 'Gain=1 (0dB)', '#CC0000')]:
    ax1.axhline(y=gain_dB, color=color, linestyle=':', alpha=0.7, label=label)
    # Find bandwidth (where A_dB crosses gain_dB)
    idx = np.argmin(np.abs(A_dB - gain_dB))
    ax1.plot(f[idx], A_dB[idx], 'o', color=color, markersize=8)
    ax1.annotate(f'BW={f[idx]:.0f} Hz' if f[idx] < 1000 else f'BW={f[idx]/1000:.0f} kHz',
                 (f[idx], gain_dB),
                 textcoords='offset points', xytext=(15, 5), fontsize=9, color=color)

ax1.set_ylabel('Gain (dB)', fontsize=12)
ax1.set_title('LM358 Open-Loop Gain (Bode Plot)', fontsize=14)
ax1.legend(fontsize=9, loc='upper right')
ax1.grid(True, which='both', alpha=0.3)
ax1.set_ylim(-20, 110)

# Phase plot
ax2.semilogx(f, phase, 'r-', linewidth=2)
ax2.axhline(y=-45, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=-90, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Frequency (Hz)', fontsize=12)
ax2.set_ylabel('Phase (degrees)', fontsize=12)
ax2.set_title('Phase Response', fontsize=13)
ax2.grid(True, which='both', alpha=0.3)
ax2.set_ylim(-100, 10)

plt.tight_layout()
plt.show()

print(f'Key LM358 frequency specs:')
print(f'  Open-loop gain (DC):    {20*np.log10(A_OL_DC):.0f} dB ({A_OL_DC:,} V/V)')
print(f'  Dominant pole:          {f_pole:.0f} Hz')
print(f'  Gain-bandwidth product: {GBW/1e6:.1f} MHz')
print(f'  Unity-gain bandwidth:   {GBW/1e6:.1f} MHz')

### Understanding the Bode Plot

The Bode plot is one of the most important graphs in any op-amp datasheet.
Here is how to read it:

1. **DC Gain**: At low frequencies, the gain is constant at ~100 dB
2. **Dominant pole**: Around 10 Hz, the gain starts rolling off at -20 dB/decade
3. **Unity-gain frequency**: Where the curve crosses 0 dB (~1 MHz for LM358)
4. **Gain-bandwidth product (GBW)**: The product of gain and bandwidth is constant
   on the -20 dB/decade slope. For LM358: GBW = 1 MHz

**Practical rule**: If you set a closed-loop gain of $G$, your bandwidth is:

$$BW = \frac{GBW}{G}$$

Examples for LM358 (GBW = 1 MHz):
- Gain = 1: BW = 1 MHz
- Gain = 10: BW = 100 kHz
- Gain = 100: BW = 10 kHz
- Gain = 1000: BW = 1 kHz

### Common Mode Rejection Ratio (CMRR)

**Typical: 80 dB**

CMRR measures how well the op-amp rejects signals that are common to both inputs.
80 dB means common-mode signals are attenuated by a factor of 10,000.

This matters when you are measuring small differential signals (like a sensor)
in the presence of large common-mode noise (like 60 Hz from power lines).

### Output Voltage Swing

**Single supply (Vcc = 5V):**
- Low: ~0V (within ~20 mV of GND when sinking current)
- High: ~Vcc - 1.5V = **3.5V** (cannot reach the positive rail!)

This is a major limitation of the LM358: the output cannot swing to the
positive supply rail. For rail-to-rail output, use a different op-amp
(like LMV324 or MCP6002).

### Slew Rate

**Typical: 0.3 V/us**

This is how fast the output can change. If you apply a step input,
the output ramps at 0.3 V per microsecond — not instantly.

For a sine wave output, slew rate limits the maximum frequency at full swing:

$$f_{max} = \frac{SR}{2\pi \cdot V_{peak}}$$

In [ ]:
# --- Output Voltage Swing Limitations ---

Vcc_values = np.linspace(3, 30, 200)

# LM358 output swing
V_out_high = Vcc_values - 1.5    # Cannot reach Vcc (BJT output stage)
V_out_low = np.full_like(Vcc_values, 0.02)  # Can get close to GND

# Rail-to-rail op-amp for comparison
V_out_high_rr = Vcc_values - 0.05  # Rail-to-rail: within ~50mV of supply
V_out_low_rr = np.full_like(Vcc_values, 0.05)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Output swing vs supply
ax1.plot(Vcc_values, Vcc_values, 'k--', linewidth=1, alpha=0.5, label='Ideal (Vcc)')
ax1.plot(Vcc_values, V_out_high, 'b-', linewidth=2, label='LM358 max output')
ax1.plot(Vcc_values, V_out_high_rr, 'g-', linewidth=2, label='Rail-to-rail max output')
ax1.fill_between(Vcc_values, V_out_high, Vcc_values, alpha=0.1, color='red',
                 label='LM358 dead zone')

ax1.set_xlabel('Supply Voltage Vcc (V)', fontsize=12)
ax1.set_ylabel('Maximum Output Voltage (V)', fontsize=12)
ax1.set_title('Output Swing: LM358 vs Rail-to-Rail', fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Slew rate effect on output
SR = 0.3e6        # 0.3 V/us = 0.3e6 V/s
t = np.linspace(0, 20e-6, 1000)  # 20 us

# Step input
V_step = np.where(t > 2e-6, 3.0, 0.0)

# Slew-rate limited output
V_out_slew = np.zeros_like(t)
for i in range(1, len(t)):
    dt = t[i] - t[i-1]
    diff = V_step[i] - V_out_slew[i-1]
    max_change = SR * dt
    if abs(diff) > max_change:
        V_out_slew[i] = V_out_slew[i-1] + np.sign(diff) * max_change
    else:
        V_out_slew[i] = V_step[i]

ax2.plot(t * 1e6, V_step, 'b--', linewidth=1.5, label='Ideal output (step)', alpha=0.7)
ax2.plot(t * 1e6, V_out_slew, 'r-', linewidth=2, label=f'LM358 (SR = {SR/1e6:.1f} V/us)')

# Annotate slew rate
ax2.annotate('Slew rate\nlimiting', (7, 1.5),
             fontsize=10, color='red',
             arrowprops=dict(arrowstyle='->', color='red'),
             xytext=(10, 0.8))

ax2.set_xlabel('Time (us)', fontsize=12)
ax2.set_ylabel('Output Voltage (V)', fontsize=12)
ax2.set_title('Slew Rate Limiting (Step Response)', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Max frequency for full-swing sine
V_peak = 2.0  # V
f_max = SR / (2 * np.pi * V_peak)
print(f'Slew rate: {SR/1e6:.1f} V/us')
print(f'Max undistorted frequency at {V_peak}V peak: {f_max/1e3:.1f} kHz')
print(f'Max undistorted frequency at {V_peak/2}V peak: {f_max*2/1e3:.1f} kHz')

---
## Section 4: Application Circuits

### Non-Inverting Amplifier

```
  V_in ──────────┬──┤+        │
                 │  │  LM358  ├──┬── V_out
                 │  ┤-        │  │
                 │  └─────────┘  │
                 │               │
                 │    ┌──[Rf]───┘
                 │    │
                 │    ├──── feedback point
                 │    │
                 │    └──[R1]──── GND
```

$$G = 1 + \frac{R_f}{R_1}$$

### Inverting Amplifier

```
  V_in ──[Ri]──┬──┤-        │
               │  │  LM358  ├──┬── V_out
               │  ┤+        │  │
     ┌──[Rf]───┘  └─────────┘  │
     │                          │
     └──────────────────────────┘
     
  (+) input tied to GND (or Vcc/2 for single supply)
```

$$G = -\frac{R_f}{R_i}$$

### Voltage Follower (Buffer)

```
  V_in ──────────┬──┤+        │
                 │  │  LM358  ├──┬── V_out = V_in
                 │  ┤-        │  │
                 │  └─────────┘  │
                 │               │
                 └───────────────┘   (output connected directly to - input)
```

Gain = 1 (unity). High input impedance, low output impedance.
Used to buffer a high-impedance source (like a voltage divider or sensor)
so it can drive a load without loading down the source.

In [ ]:
# --- Closed-Loop Gain Calculator ---

print('Op-Amp Gain Formulas')
print('=' * 60)

# Non-inverting amplifier
print('\nNON-INVERTING AMPLIFIER: G = 1 + Rf/R1')
print('-' * 40)
configs_ni = [
    (10e3, 1e3, 'Rf=10k, R1=1k'),
    (47e3, 4.7e3, 'Rf=47k, R1=4.7k'),
    (100e3, 10e3, 'Rf=100k, R1=10k'),
    (99e3, 1e3, 'Rf=99k, R1=1k'),
]
for Rf, R1, desc in configs_ni:
    G = 1 + Rf / R1
    G_dB = 20 * np.log10(G)
    BW = 1e6 / G  # LM358 GBW = 1 MHz
    print(f'  {desc:20s} -> G = {G:6.1f} ({G_dB:.1f} dB), BW = {BW/1e3:.1f} kHz')

# Inverting amplifier
print('\nINVERTING AMPLIFIER: G = -Rf/Ri')
print('-' * 40)
configs_inv = [
    (10e3, 1e3, 'Rf=10k, Ri=1k'),
    (10e3, 10e3, 'Rf=10k, Ri=10k'),
    (47e3, 10e3, 'Rf=47k, Ri=10k'),
    (100e3, 1e3, 'Rf=100k, Ri=1k'),
]
for Rf, Ri, desc in configs_inv:
    G = -Rf / Ri
    G_dB = 20 * np.log10(abs(G))
    # Noise gain for inverting = 1 + Rf/Ri
    noise_gain = 1 + Rf / Ri
    BW = 1e6 / noise_gain
    print(f'  {desc:20s} -> G = {G:6.1f} ({G_dB:.1f} dB), BW = {BW/1e3:.1f} kHz')

---
## Worked Example: Non-Inverting Amplifier with Gain of 10

**Goal:** Design a non-inverting amplifier with gain of 10 using the LM358,
powered from a single 5V supply.

### Step 1: Choose resistors

$$G = 1 + \frac{R_f}{R_1} = 10$$

$$\frac{R_f}{R_1} = 9$$

Choose R1 = 1k, then Rf = 9k. Since 9k is not a standard value,
use Rf = 9.1k (E96 series) or 10k (close enough: gain = 11).

Better option: R1 = 10k, Rf = 91k (gain = 10.1) or R1 = 1k, Rf = 8.2k + 820 ohm in series.

### Step 2: Check bandwidth

$$BW = \frac{GBW}{G} = \frac{1\text{ MHz}}{10} = 100\text{ kHz}$$

### Step 3: Check output swing

With 5V supply, max output = 5V - 1.5V = **3.5V**.
Min output = ~0V. So output range is 0 to 3.5V.

With gain of 10, the maximum input before clipping is:
$$V_{in,max} = \frac{3.5}{10} = 0.35\text{ V}$$

### Step 4: Check slew rate

At 3.5V peak output, the max undistorted frequency is:
$$f_{max} = \frac{0.3 \times 10^6}{2\pi \times 3.5} = 13.6\text{ kHz}$$

Both bandwidth (100 kHz) and slew rate (13.6 kHz at full swing) limit
the useful frequency range. The slew rate is the tighter constraint
at full output swing.

In [ ]:
# --- Design Exercise: Calculate resistor values for desired gain ---

print('Design Exercise: Non-Inverting Amplifier Resistor Selection')
print('=' * 60)

# Standard E24 resistor values (ohms)
e24_values = []
e24_base = [1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0,
             3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1]
for decade in [100, 1e3, 10e3, 100e3]:
    for val in e24_base:
        e24_values.append(val * decade)

desired_gains = [2, 5, 10, 20, 50, 100]
Vcc = 5.0
GBW = 1e6
SR = 0.3e6

print(f'\nSupply: {Vcc}V | GBW: {GBW/1e6:.0f} MHz | Slew Rate: {SR/1e6:.1f} V/us')
print(f'Max output swing: ~{Vcc-1.5:.1f}V (LM358 limitation)')
print()
print(f'{"Gain":>6} | {"R1":>8} | {"Rf":>8} | {"Actual G":>10} | {"BW":>10} | {"Max Vin":>8}')
print('-' * 70)

for G_target in desired_gains:
    ratio_needed = G_target - 1  # Rf/R1

    # Find best E24 pair
    best_err = 999
    best_R1, best_Rf = 0, 0
    for R1 in e24_values:
        Rf_ideal = R1 * ratio_needed
        # Find closest E24
        for Rf in e24_values:
            G_actual = 1 + Rf / R1
            err = abs(G_actual - G_target)
            # Prefer resistors in 1k-100k range
            if err < best_err and 1e3 <= R1 <= 100e3 and 1e3 <= Rf <= 1e6:
                best_err = err
                best_R1 = R1
                best_Rf = Rf

    G_actual = 1 + best_Rf / best_R1
    BW = GBW / G_actual
    V_max_out = Vcc - 1.5
    V_max_in = V_max_out / G_actual

    def fmt_r(r):
        if r >= 1e6:
            return f'{r/1e6:.1f}M'
        elif r >= 1e3:
            return f'{r/1e3:.1f}k'
        else:
            return f'{r:.0f}'

    print(f'{G_target:>6} | {fmt_r(best_R1):>8} | {fmt_r(best_Rf):>8} | '
          f'{G_actual:>10.2f} | {BW/1e3:>7.1f} kHz | {V_max_in*1000:>5.0f} mV')

---
## Section 5: Characteristic Curves in the Datasheet

Op-amp datasheets include many graphs. Here are the ones to pay attention to:

| Graph | What it tells you |
|-------|-------------------|
| **Open-loop gain vs frequency** | Bandwidth at any gain setting |
| **Output voltage swing vs supply** | How close output gets to rails |
| **Output voltage swing vs load** | Output drops with heavy loads |
| **Input offset voltage vs temperature** | Drift over temperature |
| **Supply current vs supply voltage** | Power consumption |
| **CMRR vs frequency** | Common-mode rejection degrades at high frequency |
| **PSRR vs frequency** | Power supply rejection degrades at high frequency |
| **Input common-mode range vs supply** | What input voltages are safe |

**Reading tip:** Always check the axes carefully. Logarithmic scales are common.
Pay attention to whether parameters are "typical" or "min/max" — design to
worst-case specs, not typical.

In [ ]:
# --- Summary: LM358 Key Parameters at a Glance ---

fig, ax = plt.subplots(figsize=(12, 7))
ax.axis('off')

# Build summary table
params = [
    ['Parameter', 'Min', 'Typical', 'Max', 'Unit'],
    ['Supply Voltage (Vcc)', '3', '5–15', '32', 'V'],
    ['Input Offset Voltage', '—', '2', '7', 'mV'],
    ['Input Bias Current', '—', '20', '150', 'nA'],
    ['Open-Loop Gain', '25', '100', '—', 'dB (V/V)'],
    ['CMRR', '65', '80', '—', 'dB'],
    ['Gain-Bandwidth Product', '—', '1.0', '—', 'MHz'],
    ['Slew Rate', '—', '0.3', '—', 'V/us'],
    ['Output Swing (high)', '—', 'Vcc-1.5', '—', 'V'],
    ['Output Swing (low)', '—', '20', '—', 'mV'],
    ['Supply Current (per amp)', '—', '0.5', '1.2', 'mA'],
    ['Output Short Circuit Current', '—', '40', '—', 'mA'],
]

table = ax.table(cellText=params[1:], colLabels=params[0],
                  cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.0, 1.6)

# Style header
for col in range(5):
    table[0, col].set_facecolor('#2E4057')
    table[0, col].set_text_props(color='white', fontweight='bold')

# Alternate row colors
for row in range(1, len(params)):
    for col in range(5):
        if row % 2 == 0:
            table[row, col].set_facecolor('#E8EEF4')

ax.set_title('LM358 Dual Op-Amp — Key Electrical Characteristics\n(at Vcc = 5V, T = 25C)',
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

---
## Simulation — Build It in Falstad

### Non-Inverting Amplifier

**What to build:**
1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an op-amp: right-click → **Draw** → **Active Components** → **Add Op Amp**
3. Connect the input signal (AC source, 0.5V, 1 kHz) to the non-inverting (+) input
4. Connect a feedback network: 10 kΩ from output to inverting (-) input, 1 kΩ from inverting input to ground
5. Add power supply connections (if required by the simulator)

**What to observe:**
- Right-click the input → **View in New Scope**, and the output → **View in New Scope**
- Gain = 1 + R_f/R_g = 1 + 10k/1k = 11. The output should be 11× the input amplitude.
- Change R_f to 4.7 kΩ — gain drops to ~5.7×. This is how you design amplifier gain with the datasheet's gain-bandwidth product in mind.

### Voltage Follower (Unity-Gain Buffer)

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an op-amp, wire the output directly to the inverting (-) input
3. Apply a signal to the non-inverting (+) input

**What to observe:**
- Output follows input exactly (gain = 1) — but the op-amp can supply much more current than the source
- This is the most common op-amp circuit for impedance buffering

### Comparator

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an op-amp with **no feedback** — leave the output disconnected from the inputs
3. Apply a sine wave to the non-inverting (+) input and a DC reference voltage (e.g. 0V) to the inverting (-) input

**What to observe:**
- The output slams to the positive or negative supply rail depending on which input is larger — there's no linear region
- This is the op-amp used as a comparator: it answers "is V+ greater than V-?" with a digital-like output

---
## Datasheet Connection — Summary

Here is a checklist for reading ANY op-amp datasheet:

**First glance (30 seconds):**
1. Single or dual supply? What voltage range?
2. How many op-amps in the package?
3. Pinout — where does everything connect?

**Quick scan (2 minutes):**
4. Open-loop gain and GBW — is it fast enough for your application?
5. Output swing — can it reach the voltages you need?
6. Slew rate — can it handle your signal frequency?
7. Supply current — does it matter for battery-powered applications?

**Deep dive (when needed):**
8. Input offset voltage — for precision DC measurements
9. Input bias current — for high-impedance sources
10. CMRR and PSRR — for noisy environments
11. Temperature coefficients — for outdoor or high-temp applications
12. Noise specs — for sensitive analog measurements

**Common LM358 alternatives:**

| Need | Better choice | Why |
|------|--------------|-----|
| Rail-to-rail output | MCP6002, LMV324 | Output swings to within mV of rails |
| Higher bandwidth | TL072 (3 MHz), OPA2134 (8 MHz) | Faster GBW |
| Lower noise | OPA2134, NE5532 | Lower voltage noise density |
| Lower offset | OPA2277 | V_OS < 20 uV |
| Lower power | MCP6002 (100 uA) | Battery-powered applications |

---
## Checkpoint Questions

1. **What is the LM358's open-loop gain at DC? How about at 10 kHz?
   (Use the Bode plot to estimate.)**

2. **You build a non-inverting amplifier with Rf = 47k and R1 = 4.7k.
   What is the gain? What is the bandwidth?**

3. **With a 5V supply, the LM358 output cannot exceed ~3.5V. If your
   amplifier has gain = 10, what is the maximum input voltage before
   the output clips?**

4. **Your sensor outputs 0–100 mV. You want to amplify it to 0–3.0V.
   What gain do you need? Choose standard resistor values for a
   non-inverting amplifier.**

5. **The LM358 has a slew rate of 0.3 V/us. You are generating a 2V
   peak-to-peak sine wave. What is the maximum frequency before
   slew-rate distortion occurs?**

6. **Why is the output voltage swing of the LM358 limited to Vcc - 1.5V?
   (Hint: think about the output stage transistor type.)**

7. **A datasheet says "typical" offset voltage is 2 mV and "max" is 7 mV.
   Which value should you use when designing a circuit? Why?**

In [ ]:
# --- Checkpoint calculation helpers ---

# Q1: Gain at 10 kHz
f_target = 10e3
A_at_f = A_OL_DC / np.sqrt(1 + (f_target / f_pole)**2)
A_at_f_dB = 20 * np.log10(A_at_f)
print(f'Q1: At DC:    A = {A_OL_DC:,.0f} V/V = {20*np.log10(A_OL_DC):.0f} dB')
print(f'    At 10kHz: A = {A_at_f:,.0f} V/V = {A_at_f_dB:.0f} dB')
print()

# Q2: Non-inverting gain and bandwidth
Rf, R1 = 47e3, 4.7e3
G = 1 + Rf / R1
BW = 1e6 / G
print(f'Q2: G = 1 + {Rf/1e3:.0f}k/{R1/1e3:.1f}k = {G:.0f}')
print(f'    BW = 1 MHz / {G:.0f} = {BW/1e3:.1f} kHz')
print()

# Q4: Sensor amplifier design
V_in_max = 0.100
V_out_target = 3.0
G_needed = V_out_target / V_in_max
print(f'Q4: G = {V_out_target}/{V_in_max} = {G_needed:.0f}')
print(f'    Rf/R1 = {G_needed - 1:.0f}')
print(f'    Use R1 = 1k, Rf = 29k (use 27k + 2.2k in series, or 30k if available)')
print(f'    Or: R1 = 3.3k, Rf = 91k (G = 1 + 91/3.3 = {1+91/3.3:.1f})')
print()

# Q5: Slew rate frequency limit
SR = 0.3e6  # V/s
V_pk = 1.0  # 2 Vpp = 1V peak
f_max = SR / (2 * np.pi * V_pk)
print(f'Q5: V_peak = 1.0V (2 Vpp)')
print(f'    f_max = {SR/1e6} V/us / (2*pi*{V_pk}V) = {f_max/1e3:.1f} kHz')